# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download abaixo da aula (se você puxar os dados direto do Kaggle pode ser que encontre resultados diferentes dos meus, afinal as bases de dados podem ter sido atualizadas).

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

### Importar Bibliotecas e Bases de Dados

In [68]:
import pandas as pd
from pathlib import Path
import re
import numpy as np

pasta_dados = Path(r'C:\Users\mobishopgamer\Documents\RepositorioGitHub\Ciencia_de_Dados\Arquivos\dados')

dados_df = pd.DataFrame()

for arquivo in pasta_dados.iterdir():
    # Criando nome do mes e do ano
    mes = arquivo.name[:3] # 3 primeiras letras no nome do arquivo
    ano = re.findall(r'\d+', arquivo.name)[0] # encontrando sequencia de numeros no nome do arquivo e selecionando a primeira sequencia

    # Criando df de cada arquivo
    arquivo_df = pd.read_csv(arquivo)

    # Adc coluna com nome do mes e coluna com nome do ano
    arquivo_df['mes'] = mes
    arquivo_df['ano'] = ano

    # Juntando todos os df em um só
    dados_df = dados_df.append(arquivo_df)


C:\Users\mobishopgamer\AppData\Local\Temp\ipykernel_14228\1226340765.py:16: DtypeWarning: Columns (62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  arquivo_df = pd.read_csv(arquivo)
C:\Users\mobishopgamer\AppData\Local\Temp\ipykernel_14228\1226340765.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_df = dados_df.append(arquivo_df)
C:\Users\mobishopgamer\AppData\Local\Temp\ipykernel_14228\1226340765.py:16: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  arquivo_df = pd.read_csv(arquivo)
C:\Users\mobishopgamer\AppData\Local\Temp\ipykernel_14228\1226340765.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_df = dados_df.append(arquivo_df)
C:\Users\mobishopgamer\AppData\Local\Temp\ipykernel_14228\1226340

### Consolidar Base de Dados

In [51]:
# Criando uma planilha com 1000 linhas para analisar qualitativa em excel

dados_df.head(1000).to_csv('primeira_analise.csv', sep=';')

# Lembrando que o objetivo é julgar o valor do AirBnB, ou seja, podemos retirar as colunas que não vão impactgar no preço, como:
# - valores iguais ou parecidos em outra coluna
# - valores onde a maioria é nulo
# - textos desnecessários


### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

In [69]:
# Colunas relevantes selecionadas

col_relevantes =['host_response_time','host_response_rate','host_is_superhost','host_total_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','weekly_price','monthly_price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','mes','ano']

dados_df = dados_df.loc[:, col_relevantes]
display(dados_df)

,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,mes,ano
0,NaN,NaN,f,1.0,-22.946854,-43.182737,Apartment,Entire home/apt,4,1.0,...,9.0,9.0,9.0,9.0,9.0,f,f,strict_14_with_grace_period,abr,2018
1,within an hour,100%,t,2.0,-22.965919,-43.178962,Condominium,Entire home/apt,5,1.0,...,9.0,10.0,10.0,9.0,9.0,t,f,strict,abr,2018
2,within an hour,100%,f,1.0,-22.985698,-43.201935,Apartment,Entire home/apt,2,1.0,...,10.0,10.0,10.0,10.0,9.0,f,f,strict,abr,2018
3,within an hour,100%,f,3.0,-22.977117,-43.190454,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,9.0,f,f,strict,abr,2018
4,within an hour,100%,t,1.0,-22.983024,-43.214270,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,9.0,t,f,strict,abr,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34324,within an hour,93%,f,0.0,-23.003180,-43.342840,Apartment,Entire home/apt,4,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,set,2019
34325,NaN,NaN,f,0.0,-22.966640,-43.393450,Apartment,Entire home/apt,4,2.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,set,2019
34326,within a few hours,74%,f,32.0,-22.962080,-43.175520,Apartment,Entire home/apt,5,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,strict_14_with_grace_period,set,2019
34327,NaN,NaN,f,0.0,-23.003400,-43.341820,Apartment,Entire home/apt,4,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,strict_14_with_grace_period,set,2019


### Tratar Valores Faltando

In [70]:
# Descobrindo quantidade de valores NaN por colunas
print(dados_df.isnull().sum())

host_response_time             401843
host_response_rate             401846
host_is_superhost                 460
host_total_listings_count         460
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                        1724
bedrooms                          850
beds                             2502
bed_type                            0
amenities                           0
price                               0
weekly_price                   826378
monthly_price                  818908
security_deposit               421280
cleaning_fee                   313506
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
number_of_reviews                   0
review_scores_rating           448016
review_scores_accuracy         448586
review_score

In [71]:
# Escluindo linhas onde temos mais de 300mil valores NaN
for col in dados_df:
    if dados_df[col].isnull().sum() > 300000:
        dados_df = dados_df.drop(col, axis=1)

print(dados_df.isnull().sum())
print(dados_df.shape) # mostra a quantidade de linhas e quantidade de colunas

host_is_superhost             460
host_total_listings_count     460
latitude                        0
longitude                       0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                    1724
bedrooms                      850
beds                         2502
bed_type                        0
amenities                       0
price                           0
guests_included                 0
extra_people                    0
minimum_nights                  0
maximum_nights                  0
number_of_reviews               0
instant_bookable                0
is_business_travel_ready        0
cancellation_policy             0
mes                             0
ano                             0
dtype: int64
(902210, 23)


In [72]:
# Analisar se a quantidade de linhas NaN que restaram serão significativas
# Como a qntd de linhas NaN que restaram são poucas comparadas ao número total de linhas, vamos remover elas também

dados_df = dados_df.dropna()

print(dados_df.shape)

(897709, 23)


### Verificar Tipos de Dados em cada coluna

In [73]:
# Conferindo se o valor na priemira linha tem coesão com o tipo de dado da coluna

print(dados_df.dtypes)

host_is_superhost             object
host_total_listings_count    float64
latitude                     float64
longitude                    float64
property_type                 object
room_type                     object
accommodates                   int64
bathrooms                    float64
bedrooms                     float64
beds                         float64
bed_type                      object
amenities                     object
price                         object
guests_included                int64
extra_people                  object
minimum_nights                 int64
maximum_nights                 int64
number_of_reviews              int64
instant_bookable              object
is_business_travel_ready      object
cancellation_policy           object
mes                           object
ano                           object
dtype: object


In [74]:
print(dados_df.iloc[0])

host_is_superhost                                                            f
host_total_listings_count                                                  1.0
latitude                                                            -22.946854
longitude                                                           -43.182737
property_type                                                        Apartment
room_type                                                      Entire home/apt
accommodates                                                                 4
bathrooms                                                                  1.0
bedrooms                                                                   0.0
beds                                                                       2.0
bed_type                                                              Real Bed
amenities                    {TV,Internet,"Air conditioning",Kitchen,Doorma...
price                                               

In [78]:
# Convertendo os tipos de dados não coesos

# price
dados_df['price'] = dados_df['price'].str.replace('$', '')
dados_df['price'] = dados_df['price'].str.replace(',', '')
dados_df['price'] = dados_df['price'].astype(np.float32, copy=False)

# extra_people
dados_df['price'] = dados_df['price'].str.replace('$', '')
dados_df['price'] = dados_df['price'].str.replace(',', '')
dados_df['price'] = dados_df['price'].astype(np.float32, copy=False)

# ano
dados_df['ano'] = dados_df['ano'].astype(np.int32)

tipos = dados_df.dtypes

for col in dados_df.columns:
    if dados_df[col].dtype == 'float64':
        dados_df[col] = dados_df[col].astype(np.float32, copy=False)
    elif dados_df[col].dtype == 'int64':
        dados_df[col] = dados_df[col].astype(np.int32, copy=False)

print(dados_df.dtypes)

host_is_superhost             object
host_total_listings_count    float32
latitude                     float32
longitude                    float32
property_type                 object
room_type                     object
accommodates                   int32
bathrooms                    float32
bedrooms                     float32
beds                         float32
bed_type                      object
amenities                     object
price                        float32
guests_included                int32
extra_people                  object
minimum_nights                 int32
maximum_nights                 int32
number_of_reviews              int32
instant_bookable              object
is_business_travel_ready      object
cancellation_policy           object
mes                           object
ano                            int32
dtype: object


### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo